## Details

#### Name: Akshat Toolaj Sinha
#### Entry Number: 2020CSB1068
#### Equity: Tech Mahindra

## Libraries & Constants

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math 
import yfinance as yf
import datetime
import os
import matplotlib.dates as mdates
from arch import arch_model
from scipy.stats import norm

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
sns.set_palette('flare')
sns.set_style("darkgrid")
sns.despine()


In [ ]:
TICKER='TECHM.NS'
PERIOD='max'
FILE_NAME='TECH_MAHINDRA.csv'
PRICE_ANALYSIS='Close'
EQUITY_NAME='Tech Mahindra'
SIGNIFICANCE_LEVEL=0.05
TRADING_DAYS=252
YEAR_DAYS=365
OPTION_EXPIRY=datetime.date(2024,5,31)
TODAY=datetime.date.today()

# Risk Free Rate for 91 Days
RISK_FREE_RATE= 6.88

## Data Downloading & Augmentation

In [ ]:
Equity_df=yf.download(TICKER,period=PERIOD,auto_adjust=True)

In [ ]:
Equity_df.reset_index(inplace=True)
Equity_df = Equity_df.round(4)

In [ ]:
Equity_df.to_csv(FILE_NAME,index=False)

## Data Visualization

Plotting Equity price 

In [ ]:
sns.lineplot(data=Equity_df,x='Date',y=PRICE_ANALYSIS)
plt.xlabel("Year")
plt.ylabel("Price (Rs.)")
plt.title(EQUITY_NAME+ " Equity Price")
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)
plt.yticks(range(0,int(max(Equity_df[PRICE_ANALYSIS]))+100,100))
plt.show()

Plotting Log Returns

In [ ]:
log_returns=np.log(Equity_df[PRICE_ANALYSIS]/Equity_df[PRICE_ANALYSIS].shift(1))

In [ ]:
log_returns=log_returns.dropna()
log_returns.reset_index(drop=True,inplace=True)

In [ ]:
log_returns=log_returns*100

In [ ]:
dates=Equity_df['Date'][1:]

In [ ]:
sns.lineplot(x=dates,y=log_returns)
plt.xlabel("Year")
plt.ylabel("Returns (%)")
plt.title(EQUITY_NAME+" Daily Returns")
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)
plt.yticks(np.arange(int(min(log_returns))-1,max(log_returns)+1,2))
plt.show()

## Normality Tests

In [ ]:
from statsmodels.graphics.gofplots import qqplot

qqplot(log_returns,line='s')
plt.title("QQ Plot of Log Returns")
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Sample Quantiles")
plt.show()

In [ ]:
sns.histplot(log_returns,kde=True)

In [ ]:
print("Mean of Log Returns: ",round(log_returns.mean(),4))
print("Standard Deviation of Log Returns: ",round(log_returns.std(),4))
print("Skewness of Log Returns: ",round(log_returns.skew(),4))
print("Kurtosis of Log Returns: ",round(log_returns.kurtosis(),4))

In [ ]:
# Jarque-Bera Test
from scipy.stats import jarque_bera


jb_test=jarque_bera(log_returns)
print("Jarque-Bera Test Statistic: ",jb_test[0])
print("Jarque-Bera Test P-Value: ",jb_test[1])

if jb_test[1]<SIGNIFICANCE_LEVEL:
    print("\nReject Null Hypothesis: The data is not normally distributed")
else:
    print("\nFail to Reject Null Hypothesis : The data is normally distributed")

In [ ]:
# use Kolmogorov-Smirnov test to check normality

from scipy.stats import kstest

ks_test=kstest(log_returns,'norm')

print("\nKolmogorov-Smirnov Test Statistic: ",ks_test.statistic)
print("Kolmogorov-Smirnov Test P-Value: ",ks_test.pvalue)

if ks_test.pvalue<SIGNIFICANCE_LEVEL:
    print("\nReject Null Hypothesis: The data is not normally distributed")
else:
    print("\nFail to Reject Null Hypothesis : The data is normally distributed")

In [ ]:
# use Shapiro-Wilk test to check normality

from scipy.stats import shapiro

shapiro_test=shapiro(log_returns)

print("\nShapiro-Wilk Test Statistic: ",shapiro_test[0])
print("Shapiro-Wilk Test P-Value: ",shapiro_test[1])

if shapiro_test[1]<SIGNIFICANCE_LEVEL:
    print("\nReject Null Hypothesis: The data is not normally distributed")
else:
    print("\nFail to Reject Null Hypothesis : The data is normally distributed")

## Returns Analysis

In [ ]:
# Check for auto correlation in log_returns

import statsmodels.api as sm

sm.graphics.tsa.plot_acf(log_returns,lags=20)
plt.xlabel('Lag')
plt.ylabel("Auto-Correlation")
plt.title("Auto-Correlation Function (ACF) of Log Returns")
plt.ylim(-0.3,1.1)
plt.yticks(np.arange(-0.3,1.1,0.05))
plt.show()



## Volatility Modelling

In [ ]:
HistoricalVolatility=np.std(log_returns)
AnnualHistoricalVolatility=HistoricalVolatility*math.sqrt(TRADING_DAYS)
print("\nHistorical Daily Volatility (%): ",HistoricalVolatility)
print("Historical Annual Volatility (%): ",AnnualHistoricalVolatility)

In [ ]:
CandidateModels=[(1,1),(1,2),(2,1),(2,2),(3,2),(2,3),(3,3)]

BestModel=None
BestModelStatistic= float('-inf')
Parameters=None

for p,q in CandidateModels:
    model=arch_model(log_returns,vol='Garch',p=p,q=q)
    model_fit=model.fit(disp='off')
    if model_fit.aic>BestModelStatistic:
        BestModelStatistic=model_fit.aic
        BestModel=model_fit
        Parameters=(p,q)

print("\nBest Parameters(p,q): ",Parameters)

GARCHVolatility=BestModel.forecast(horizon=1).variance.iloc[-1].values[-1]
AnnualGARCHVolatility=GARCHVolatility*math.sqrt(TRADING_DAYS)

print("\nGARCH Daily Volatility (%): ",GARCHVolatility)
print("GARCH Annual Volatility (%): ",AnnualGARCHVolatility)

## Option Pricing

In [ ]:
def nCr(n,r):
    f = math.factorial
    return f(n)/(f(r)*f(n-r))

In [ ]:
from abc import ABC,abstractmethod

class OptionPricing(ABC):

    @abstractmethod
    def __init__(self,spot_price,strike_price,risk_free_rate,volatility,time_to_expiry):
        pass

    @abstractmethod
    def OptionPrice(self,type):
        pass

    @abstractmethod
    def setStrikePrice(self,strikePrice):
        pass


In [ ]:
class OptionPricingCRR(OptionPricing):

    def __init__(self,s0,Annualvolatility,strikePrice,maturity,riskFreeRate,steps,dividentYield=0):
        self.s0=s0
        self.strikePrice=strikePrice
        self.steps=steps
        self.dividentYield=dividentYield
        self.DaysToMaturity=(maturity-TODAY).days

        self.volatility=Annualvolatility*math.sqrt(self.DaysToMaturity/YEAR_DAYS)
        self.riskFreeRate=(riskFreeRate)/(YEAR_DAYS*self.steps/self.DaysToMaturity)

        self.delta=self.DaysToMaturity/self.steps
        self.u=math.exp(self.volatility*math.sqrt(self.delta))
        self.d=1/self.u
        
        self.riskNeutralProbability=(math.exp((self.riskFreeRate-self.dividentYield)*self.delta)-self.d)/(self.u-self.d)

    def OptionPrice(self,type='C'):

        FuturePrice=0

        for i in range(0,self.steps+1):
            equityPriceMaturity=self.s0*(self.u**i)*(self.d**(self.steps-i))

            if type=='C':
                profit=max(equityPriceMaturity-self.strikePrice,0)
            else:
                profit=max(self.strikePrice-equityPriceMaturity,0)

            prob=(self.riskNeutralProbability**i)*((1-self.riskNeutralProbability)**(self.steps-i))
            

            FuturePrice+=profit*nCr(self.steps,i)*prob

        discountFactor=math.exp(-self.riskFreeRate*self.steps)

        return FuturePrice*discountFactor
    

    def setStrikePrice(self,strikePrice):
        self.strikePrice=strikePrice

In [ ]:
class OptionPricingSimulation(OptionPricing):
    def __init__(self,s0,Annualvolatility,strikePrice,maturity,riskFreeRate,steps,dividentYield=0,numSimulations=100000):
        self.s0=s0
        self.strikePrice=strikePrice
        self.steps=steps
        self.dividentYield=dividentYield
        self.DaysToMaturity=(maturity-TODAY).days

        self.stepFactor=self.DaysToMaturity/self.steps

        self.volatility=Annualvolatility*math.sqrt(self.DaysToMaturity/YEAR_DAYS)*math.sqrt(self.stepFactor)
        self.riskFreeRate=(riskFreeRate/YEAR_DAYS)*self.stepFactor
        
        self.numSimulations=numSimulations

    def OptionPrice(self,type='C'):

        totalPayoff=0

        for i in range(0,self.numSimulations):

            trendTerm=(self.riskFreeRate-self.dividentYield-0.5*self.volatility**2)*self.steps
            volatilityTerm=self.volatility*math.sqrt(self.steps)*np.random.normal()

            equityPrice=self.s0*math.exp(trendTerm+volatilityTerm)

            if type=='C':
                payoff=max(equityPrice-self.strikePrice,0)
            else:
                payoff=max(self.strikePrice-equityPrice,0)

            totalPayoff+=payoff
        
        AveragePayoff=totalPayoff/self.numSimulations

        discountFactor=math.exp(-self.riskFreeRate*self.steps)

        return AveragePayoff*discountFactor

    def setStrikePrice(self,strikePrice):
        self.strikePrice=strikePrice
    

In [37]:
class OptionPricingBS(OptionPricing):

    def __init__(self,s0,Annualvolatility,strikePrice,maturity,riskFreeRate,dividentYield=0):
        self.s0=s0
        self.strikePrice=strikePrice
        self.Annualvolatility=Annualvolatility
        self.dividentYield=dividentYield
        self.YearMaturity=(maturity-TODAY).days/YEAR_DAYS
        self.riskFreeRate=riskFreeRate

    def OptionPrice(self,type='C'):
        term1=math.log(self.s0/self.strikePrice)
        term2=(self.riskFreeRate-self.dividentYield+0.5*self.Annualvolatility**2)*self.YearMaturity
        denominator=self.Annualvolatility*math.sqrt(self.YearMaturity)

        d1=(term1+term2)/denominator
        d2=d1-self.Annualvolatility*math.sqrt(self.YearMaturity)

        if type=='C':
            part1=self.s0*math.exp(-self.dividentYield*self.YearMaturity)*norm.cdf(d1)
            part2=self.strikePrice*math.exp(-self.riskFreeRate*self.YearMaturity)*norm.cdf(d2)
            return part1-part2
        else:
            part1=self.strikePrice*math.exp(-self.riskFreeRate*self.YearMaturity)*norm.cdf(-d2)
            part2=self.s0*math.exp(-self.dividentYield*self.YearMaturity)*norm.cdf(-d1)
            return part1-part2
        

    def setStrikePrice(self,strikePrice):
        self.strikePrice=strikePrice

In [38]:
CurrentPrice=Equity_df[PRICE_ANALYSIS].iloc[-1]
strikePrice=1180
RiskFreeRate=RISK_FREE_RATE/100
step=100


# Volatility=AnnualHistoricalVolatility/100
Volatility=AnnualGARCHVolatility/100

In [39]:
optionCRR=OptionPricingCRR(CurrentPrice,
                        Volatility,
                        strikePrice,
                        OPTION_EXPIRY,
                        RiskFreeRate,
                        step)

optionCRR.OptionPrice('C')

430.54411021309653

In [40]:
optionSimulation=OptionPricingSimulation(CurrentPrice,
                        Volatility,
                        strikePrice,
                        OPTION_EXPIRY,
                        RiskFreeRate,
                        step)

optionSimulation.OptionPrice('C')

430.46634470881133

In [41]:
optionBS=OptionPricingBS(CurrentPrice,
                        Volatility,
                        strikePrice,
                        OPTION_EXPIRY,
                        RiskFreeRate)

optionBS.OptionPrice('C')

89.46835523798882